In [18]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import numpy as np

class CustomFashionMNIST(Dataset):
    def __init__(self, root='./data', train=True, transform=None):
        self.dataset = datasets.FashionMNIST(
            root=root,
            train=train,
            download=True
        )
        self.images = self.dataset.data.numpy().astype(np.float32) / 255.0
        self.labels = self.dataset.targets.numpy()
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img = torch.FloatTensor(self.images[index]).unsqueeze(0)
        lbl = self.labels[index]
        if self.transform:
            img = self.transform(img)
        return img, lbl

class SimpleFashionNet(nn.Module):
    def __init__(self):
        super(SimpleFashionNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        # Updated fully connected layer with the correct input size
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def save_best_model(model, accuracy):
    state = {
        'model_state_dict': model.state_dict(),
        'accuracy': accuracy
    }
    filename = 'best_model.pt'
    torch.save(state, filename)
    print(f"Best model saved as '{filename}' with accuracy: {accuracy:.2f}%")

def load_best_model(filepath='best_model.pt'):
    """Load the saved model."""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = SimpleFashionNet().to(device)
    try:
        checkpoint = torch.load(filepath, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        accuracy = checkpoint.get('accuracy', 0.0)
        print(f"Model loaded from '{filepath}' with accuracy: {accuracy:.2f}%")
        return model, accuracy
    except FileNotFoundError:
        print(f"No saved model found at '{filepath}'")
        return model, 0.0

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Computation will run on: {device}")

    train_data = CustomFashionMNIST(train=True)
    test_data = CustomFashionMNIST(train=False)

    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=64)

    model = SimpleFashionNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    loss_fn = nn.CrossEntropyLoss()

    num_epochs = 50
    best_accuracy = 0.0

    for epoch in range(num_epochs):
        model.train()
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            if i % 50 == 0:
                print(f"Epoch {epoch + 1}/{num_epochs}, Step {i}, Loss: {loss.item():.4f}")

        model.eval()
        total, correct = 0, 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Test accuracy for Epoch {epoch + 1}: {accuracy:.2f}%")

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            save_best_model(model, best_accuracy)

    print(f"Highest accuracy: {best_accuracy:.2f}%")

Computation will run on: cpu


100%|██████████| 26.4M/26.4M [00:01<00:00, 17.6MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 309kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.70MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 13.4MB/s]


Epoch 1/50, Step 0, Loss: 2.3068
Epoch 1/50, Step 50, Loss: 0.9918
Epoch 1/50, Step 100, Loss: 0.8850
Epoch 1/50, Step 150, Loss: 0.7584
Epoch 1/50, Step 200, Loss: 0.5911
Epoch 1/50, Step 250, Loss: 0.5119
Epoch 1/50, Step 300, Loss: 0.4519
Epoch 1/50, Step 350, Loss: 0.5402
Epoch 1/50, Step 400, Loss: 0.5489
Epoch 1/50, Step 450, Loss: 0.6554
Epoch 1/50, Step 500, Loss: 0.6664
Epoch 1/50, Step 550, Loss: 0.5373
Epoch 1/50, Step 600, Loss: 0.3153
Epoch 1/50, Step 650, Loss: 0.3696
Epoch 1/50, Step 700, Loss: 0.4454
Epoch 1/50, Step 750, Loss: 0.5012
Epoch 1/50, Step 800, Loss: 0.3395
Epoch 1/50, Step 850, Loss: 0.4018
Epoch 1/50, Step 900, Loss: 0.4722
Test accuracy for Epoch 1: 84.44%
Best model saved as 'best_model.pt' with accuracy: 84.44%
Epoch 2/50, Step 0, Loss: 0.5752
Epoch 2/50, Step 50, Loss: 0.3982
Epoch 2/50, Step 100, Loss: 0.3241
Epoch 2/50, Step 150, Loss: 0.4627
Epoch 2/50, Step 200, Loss: 0.2949
Epoch 2/50, Step 250, Loss: 0.3731
Epoch 2/50, Step 300, Loss: 0.3450
Epoc